In [1]:
using RigidBodyDynamics
using MathProgBase
using ForwardDiff

In [4]:
type DynamicsFixedPointProgram{M} <: MathProgBase.AbstractNLPEvaluator
    mechanism::Mechanism{M}
#     state::MechanismState # TODO: store here
end

In [5]:
getmechanism(d::DynamicsFixedPointProgram) = d.mechanism
num_variables(d::DynamicsFixedPointProgram) = num_positions(getmechanism(d))
num_constraints(d::DynamicsFixedPointProgram) = num_velocities(getmechanism(d)) # TODO: update when necessary

num_constraints (generic function with 1 method)

In [6]:
function MathProgBase.initialize(d::DynamicsFixedPointProgram, requested_features::Vector{Symbol})
    for feat in requested_features
        if !(feat in MathProgBase.features_available(d))
            error("Unsupported feature $feat")
            # TODO: implement Jac-vec and Hess-vec products
            # for solvers that need them
        end
    end
end

MathProgBase.features_available(d::DynamicsFixedPointProgram) = [:Grad, :Jac, :Hess]

features_available (generic function with 5 methods)

In [7]:
MathProgBase.eval_f(d::DynamicsFixedPointProgram, q) = 0

function MathProgBase.eval_g(d::DynamicsFixedPointProgram, g, q)
    mechanism = getmechanism(d)
    state = MechanismState(eltype(q), mechanism)
    set_configuration!(state, q)
    zero_velocity!(state)
    q̇, v̇ = dynamics(state)
    g[:] = velocity_dict_to_vector(v̇, joints(mechanism))
    g
end

function MathProgBase.eval_grad_f(d::DynamicsFixedPointProgram, grad_f, q)
    grad_f[:] = ForwardDiff.gradient(q -> MathProgBase.eval_f(d, q), q)
    grad_f
end

function MathProgBase.jac_structure(d::DynamicsFixedPointProgram)
    # TODO: determine and exploit structure
    constraints = num_constraints(d)
    vars = num_variables(d)
    [i for i in 1 : constraints, j in 1 : vars], [j for i in 1 : constraints, j in 1 : vars]
end

function MathProgBase.hesslag_structure(d::DynamicsFixedPointProgram)
    vars = num_variables(d)
    [i for i in 1 : vars, j in 1 : vars], [j for i in 1 : vars, j in 1 : vars]
end

function MathProgBase.eval_jac_g(d::DynamicsFixedPointProgram, J, q)
    fun = q -> begin
        g = Array(eltype(q), num_variables(d))
        MathProgBase.eval_g(d, g, q)
    end
    J[:] = ForwardDiff.jacobian(fun, q)[:]
end

function MathProgBase.eval_hesslag(d::DynamicsFixedPointProgram, H, q, σ, μ)
    H[:] = σ * ForwardDiff.hessian(q -> MathProgBase.eval_f(d, q), q)[:]
    for i = 1 : num_constraints(d)
        fun = q -> begin
            g = Array(eltype(q), num_variables(d))
            MathProgBase.eval_g(d, g, q)[i]
        end
        H[:] += μ[i] * ForwardDiff.hessian(fun, q)[:]
    end
    
end

eval_hesslag (generic function with 5 methods)

In [8]:
function fixedpointtest(solver=MathProgBase.defaultNLPsolver)
    filename = "/Users/twan/code/drake-distro/drake/examples/Acrobot/Acrobot.urdf"
    mechanism = parse_urdf(Float64, filename)
    problem = DynamicsFixedPointProgram(mechanism)
    vars = num_variables(problem)
    constraints = num_constraints(problem)
    m = MathProgBase.NonlinearModel(solver)
    l = -Inf * ones(vars) # TODO: joint limits
    u = Inf * ones(vars) # TODO: joint limits
    MathProgBase.loadproblem!(m, vars, constraints, l, u, zeros(vars), zeros(vars), :Min, problem)
    MathProgBase.setwarmstart!(m, rand(vars))

    MathProgBase.optimize!(m)
    stat = MathProgBase.status(m)

#     @test stat == :Optimal
    x = MathProgBase.getsolution(m)
end

fixedpointtest (generic function with 2 methods)

In [11]:
fixedpointtest()

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

2-element Array{Float64,1}:
  3.38443e-19
 -3.14159    